# Gaussian image denoising

In [ ]:
local_debug = True

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
#from ipywidgets import *
import cv2
import time
#import kernels
import mrcfile
from collections import namedtuple

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
Args = namedtuple("args", "input")
args = Args("FoilHole.mrc")

In [ ]:
%%bash
rm {args.input}
OUTPUT_FILENAME={args.input}
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="11lmIglqAgIgTRS3YZZccDM-ma2bI-CXr"
    wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME 2> /dev/null
fi

In [ ]:
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
padded_noisy = np.zeros(shape=(noisy.shape[0]+32, noisy.shape[1]+32), dtype=np.uint8)
padded_noisy[16:-16, 16:-16] = noisy
noisy = padded_noisy

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
import denoising.image.gaussian as denoising

In [ ]:
denoiser = denoising.Monochrome_Denoising(logger)

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

sigma = np.array([1.5, 1.5])
kernel = [None]*2
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
denoised = denoiser.filter(noisy, kernel)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy, cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised, cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy, denoised)})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[200:400,800:1000], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[200:400,800:1000], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()